# Preliminary TCGA DGE analysis
### Primary concerns about this analysis:
1. Low statistical power: healthy condition has few samples
2. Missing data: ovarian cancer cohort has no healthy samples

In [1]:
library(tidyverse)
library(TCGAbiolinks)
library(DESeq2)
library(BiocParallel)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects

## Take advantage of parallelization in DESeq2 functions

In [2]:
n_cores <- detectCores()
BiocParallel::register(MulticoreParam(n_cores))

## Constants

In [3]:
projects <- c("TCGA-UCEC", "TCGA-CESC")
data_root <- "../../../../../mnt/d/TCGA"
project_dirs <- sapply(
    projects, 
    (function(p) paste0(data_root, "/", as.character(Sys.Date()), "-", p, "-", "TCGAbiolinks"))
)
tumor_levels <- c("Primary solid Tumor", "Metastatic")
healthy_levels <- c("Solid Tissue Normal")
tumor_def <- "Tumor"
healthy_def <- "Healthy"
proj_idx <- 2

## Functions

In [4]:
# Consolidate levels and (optionally) filter remaining levels
# Ex:
# - {"Primary solid Tumor", "Metastatic"} -> "Tumor"
# - {"Solid Tissue Normal"} -> "Healthy"
# - {"Recurrent Tumor"} -> (rows removed)
consolidate_levels <- function(d, old_tumor_levels, old_healthy_levels, new_tumor_level, new_healthy_level, drop_remaining = TRUE) {
    tumor_mask <- d$definition %in% old_tumor_levels
    healthy_mask <- d$definition %in% old_healthy_levels
    d$definition[tumor_mask] <- new_tumor_level
    d$definition[healthy_mask] <- new_healthy_level
    
    if (drop_remaining) {
        level_mask <- d$definition %in% c(new_tumor_level, new_healthy_level)
        d <- d[, level_mask]
    }
}


# Threshold based on expression in both conditions (assumes "Tumor"/"Healthy" are only levels)
filter_by_expression <- function(dds, tumor_level, healthy_level, min_expr) {
    tumor_cond_mask <- dds$definition == tumor_level
    healthy_cond_mask <- dds$definition == healthy_level
    tumor_cond_expr_mask <- rowSums(DESeq2::counts(dds[, tumor_cond_mask])) >= min_expr
    healthy_cond_expr_mask <- rowSums(DESeq2::counts(dds[, healthy_cond_mask])) >= min_expr
    expr_mask <- tumor_cond_expr_mask & healthy_cond_expr_mask
    return(dds[expr_mask, ])
}


# Select which rows to keep based on adjusted p-value and log2 fold-change
filter_DGE_res <- function(df, max_padj = 0.05, min_l2fc = log2(2)) {
    padj_mask <- df$padj <= max_padj
    l2fc_mask <- df$log2FoldChange >= min_l2fc
    final_mask <- padj_mask & l2fc_mask
    return(df[final_mask, ])
}

## Run Query & download

In [5]:
query <- GDCquery(
    project = projects[proj_idx],
    data.category = "Transcriptome Profiling",
    data.type = "Gene Expression Quantification",
    workflow.type = "HTSeq - Counts"
)

--------------------------------------
o GDCquery: Searching in GDC database
--------------------------------------
Genome of reference: hg38
--------------------------------------------
oo Accessing GDC. This might take a while...
--------------------------------------------
ooo Project: TCGA-CESC
--------------------
oo Filtering results
--------------------
ooo By data.type
ooo By workflow.type
----------------
oo Checking data
----------------
ooo Check if there are duplicated cases
ooo Check if there results for the query
-------------------
o Preparing output
-------------------


In [6]:
GDCdownload(query, method = "api", directory = project_dirs[proj_idx], files.per.chunk = 10)

Of the 309 files for download 309 already exist.
All samples have been already downloaded


## Prepare data for analysis

In [7]:
data <- GDCprepare(query, directory = project_dirs[proj_idx])

|====================================================|100%                      Completed after 29 s 


Starting to add information to samples
 => Add clinical information to samples
Add FFPE information. More information at: 
=> https://cancergenome.nih.gov/cancersselected/biospeccriteria 
=> http://gdac.broadinstitute.org/runs/sampleReports/latest/FPPP_FFPE_Cases.html
 => Adding subtype information to samples
cesc subtype information from:doi:10.1038/nature21386
Accessing www.ensembl.org to get gene information
Warning message:
“`select_()` is deprecated as of dplyr 0.7.0.
Please use `select()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Warning message:
“`filter_()` is deprecated as of dplyr 0.7.0.
Please use `filter()` instead.
See vignette('programming') for more help
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Cache found
From the 60483 genes we couldn't map 3990


In [8]:
data

class: RangedSummarizedExperiment 
dim: 56493 309 
metadata(1): data_release
assays(1): HTSeq - Counts
rownames(56493): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(309): TCGA-JX-A3Q8-01A-11R-A21T-07
  TCGA-C5-A1BK-01B-11R-A13Y-07 ... TCGA-EK-A2R8-01A-21R-A18M-07
  TCGA-JW-A5VK-01A-11R-A28H-07
colData names(131): sample patient ... subtype_GEXP.APOBEC3H.164668
  subtype_patient

### Only want two levels

Lump "Metastatic" and "Primary Solid Tumor" together, define "Healthy" as non-tumor, and filter out data from other levels

In [9]:
data_consolidated <- consolidate_levels(
    data,
    old_tumor_levels = tumor_levels,
    old_healthy_levels = healthy_levels,
    new_tumor_level = tumor_def,
    new_healthy_level = healthy_def,
    drop_remaining = TRUE
)

In [17]:
unique(data_consolidated$definition)
dim(data_consolidated)

[1] "Tumor"   "Healthy"

[1] 56493   309

In [11]:
sum(data_consolidated$definition == tumor_def)
sum(data_consolidated$definition == healthy_def)

[1] 306

[1] 3

## Prep data for DGE analysis

In [12]:
dds <- DESeqDataSet(data_consolidated, design = ~ definition)

renaming the first element in assays to 'counts'
converting counts to integer mode
Warning message in DESeqDataSet(data_consolidated, design = ~definition):
“some variables in design formula are characters, converting to factors”

In [13]:
dds

class: DESeqDataSet 
dim: 56493 309 
metadata(2): data_release version
assays(1): counts
rownames(56493): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(309): TCGA-JX-A3Q8-01A-11R-A21T-07
  TCGA-C5-A1BK-01B-11R-A13Y-07 ... TCGA-EK-A2R8-01A-21R-A18M-07
  TCGA-JW-A5VK-01A-11R-A28H-07
colData names(131): sample patient ... subtype_GEXP.APOBEC3H.164668
  subtype_patient

### Filter out genes which are not expressed in either tumor or healthy conditions
I.E. we want to keep only genes that are expressed (at least once) in both the healthy AND the tumor conditions

In [15]:
dds_filtered <- filter_by_expression(dds, tumor_level = tumor_def, healthy_level = healthy_def, min_expr = 1)

In [18]:
dim(dds_filtered)

[1] 35549   309

## Perform DGE analysis

In [19]:
ddsSeq <- DESeq(dds_filtered, parallel = TRUE)

estimating size factors
estimating dispersions
gene-wise dispersion estimates: 16 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 16 workers
-- replacing outliers and refitting for 3297 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


In [20]:
ddsSeq

class: DESeqDataSet 
dim: 35549 309 
metadata(2): data_release version
assays(6): counts mu ... replaceCounts replaceCooks
rownames(35549): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(26): ensembl_gene_id external_gene_name ... maxCooks
  replace
colnames(309): TCGA-JX-A3Q8-01A-11R-A21T-07
  TCGA-C5-A1BK-01B-11R-A13Y-07 ... TCGA-EK-A2R8-01A-21R-A18M-07
  TCGA-JW-A5VK-01A-11R-A28H-07
colData names(133): sample patient ... sizeFactor replaceable

In [21]:
resultsNames(ddsSeq)

[1] "Intercept"                   "definition_Tumor_vs_Healthy"

### Get DGE results using *Tumor vs. Healthy* statistics
This means that, for some $gene_a$, $\log_{2}fc = 2$ means $gene_a$ was expressed *4 times as much* in the tumor condition as in the healthy condition.

In [22]:
res <- results(ddsSeq, contrast = c("definition", "Tumor", "Healthy"), pAdjustMethod = "BH", parallel = TRUE)

## Combine results with external gene IDs

In [24]:
res_df <- as_tibble(res, rownames = "geneID")
ddsSeq_row_data_df <- rowData(ddsSeq) %>% as_tibble()

### Make sure gene ordering is preserved between dataframes

In [25]:
all(ddsSeq_row_data_df$ensembl_gene_id == res_df$geneID)

[1] TRUE

### Place all gene IDs in the same dataframe as the results statistics

In [26]:
res_genes_df <- res_df %>% mutate(
        external_gene_name = ddsSeq_row_data_df$external_gene_name,
        original_ensembl_gene_id = ddsSeq_row_data_df$original_ensembl_gene_id,
        ensembl_gene_id = res_df$geneID
    ) %>%
    select(-geneID) %>%
    select(ensembl_gene_id, original_ensembl_gene_id, external_gene_name, everything())

In [30]:
head(res_genes_df)

ensembl_gene_id,original_ensembl_gene_id,external_gene_name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003,ENSG00000000003.13,TSPAN6,3476.033426,0.3218579,0.4444177,0.7242239,4.689283e-01,6.487193e-01
ENSG00000000005,ENSG00000000005.5,TNMD,2.267135,-3.8836735,1.7357334,-2.2374828,2.525480e-02,9.194335e-02
ENSG00000000419,ENSG00000000419.11,DPM1,2638.597733,0.3850439,0.2701783,1.4251477,1.541145e-01,3.193678e-01
ENSG00000000457,ENSG00000000457.12,SCYL3,910.186204,0.2949981,0.2782027,1.0603710,2.889758e-01,4.799699e-01
ENSG00000000460,ENSG00000000460.15,C1orf112,878.032328,2.4829943,0.3161081,7.8548899,4.001234e-15,4.944582e-13
ENSG00000000938,ENSG00000000938.11,FGR,514.668255,-1.1657661,0.5908316,-1.9730936,4.848490e-02,1.465515e-01


In [31]:
dim(res_genes_df)

[1] 35549     9

## Create DEG list

### Drop rows (genes) with NA values

In [32]:
na_mask <- rowSums(is.na(res_genes_df)) > 0
res_genes_df <- res_genes_df[!na_mask, ]

No NAs left?

In [33]:
sum(rowSums(is.na(res_genes_df))) == 0

[1] TRUE

How many rows left?

In [34]:
nrow(res_genes_df)

[1] 28299

### Apply final filters (adj. $p$-values, $\log_2$ fold-change)

#### Least strict

In [45]:
deg1_padj <- 0.05
deg1_l2fc <- log2(2)
DEG1_df <- filter_DGE_res(res_genes_df, max_padj = deg1_padj, min_l2fc = deg1_l2fc)

How many rows left?

In [46]:
nrow(DEG1_df)

[1] 3393

#### More strict

In [48]:
deg2_padj <- 0.01
deg2_l2fc <- log2(4)
DEG2_df <- filter_DGE_res(res_genes_df, max_padj = deg2_padj, min_l2fc = deg2_l2fc)

How many rows left?

In [49]:
nrow(DEG2_df)

[1] 1691

#### Most strict

In [50]:
deg3_padj <- 0.005
deg3_l2fc <- log2(8)
DEG3_df <- filter_DGE_res(res_genes_df, max_padj = deg3_padj, min_l2fc = deg3_l2fc)

How many rows left?

In [51]:
nrow(DEG3_df)

[1] 1110

## Save filtered DEG lists

In [52]:
as.character(deg1_padj)

[1] "0.05"

In [59]:
DEG1_file <- paste0("DEG1", "_padj_", as.character(deg1_padj), "_l2fc_", as.character(deg1_l2fc), ".tsv")
DEG2_file <- paste0("DEG2", "_padj_", as.character(deg2_padj), "_l2fc_", as.character(deg2_l2fc), ".tsv")
DEG3_file <- paste0("DEG3", "_padj_", as.character(deg3_padj), "_l2fc_", as.character(deg3_l2fc), ".tsv")

In [60]:
DEG1_file
DEG2_file
DEG3_file

[1] "DEG1_padj_0.05_l2fc_1.tsv"

[1] "DEG2_padj_0.01_l2fc_2.tsv"

[1] "DEG3_padj_0.005_l2fc_3.tsv"

In [65]:
write_tsv(DEG1_df, path = paste(data_root, "DGE_analysis", DEG1_file, sep="/"))
write_tsv(DEG2_df, path = paste(data_root, "DGE_analysis", DEG2_file, sep="/"))
write_tsv(DEG3_df, path = paste(data_root, "DGE_analysis", DEG3_file, sep="/"))